In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

torch.manual_seed(0)

# Spiegazione
In questo file ho provato a fare un piccolo esperimento per i fatti miei. Ho voluto lavorare sul Fashion Mnist (immagini a toni di grigio 28x28) è creare un modello alternativo di 
CNN capace di fare classificazione multi-classi.

Notiamo che all'aumentare del numero di layer lineari (CLASSIFICATORE) aumenta anche il numero di epoche necessarie per l'addestramento. Le performance faranno inizialmente schifo perchè la rete tirerà completamente a caso, però dopo un po' inizia a migliorare notevolmente fino a raggiungere un overfitting se superi le 4 epoche. 
Si nota infatti che l'andamento della loss sul training è altalenante e che non si hanno miglioramenti sul test set. 

# Preparazione del modello: CNN

In [36]:
import torch.optim.sgd


class FashionCNN(nn.Module):
    def __init__(self):
        # Chiamata al costruttore della classe base:
        super(FashionCNN, self).__init__()
        # Prima convoluzione
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=3,
            kernel_size=7 ,
            stride=1,
            padding=3
        )
        # Max pooling:
        self.pooling1 = nn.MaxPool2d(
            kernel_size=2
        )
        # Seconda convoluzione
        self.conv2 = nn.Conv2d(
            in_channels=3,
            out_channels=6,
            kernel_size=5,
            stride=1,
            padding=2
        )
        # Max pooling:
        self.pooling2 = nn.MaxPool2d(
            kernel_size=2
        )
        # Linear layers:
        self.linear1 = nn.Linear(in_features=6*7*7, out_features=100)
        self.linear2 = nn.Linear(in_features=100, out_features=50)
        self.linear3 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):
        # x è un'immagine a toni di grigio (1, 28, 28)
        x = self.conv1(x)
        x = self.pooling1(x)
        x = self.conv2(x)
        x = self.pooling2(x)
        # Ora devo flattinizzare x per poter lavorare sui layer lineari:
        x = x.view(-1, 6*7*7)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.log_softmax(x, dim=1)
        return x

# Istanzio il modello, l'optimizer
model = FashionCNN()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Preparazione del dataset Fashion MNIST

In [37]:
# Mi creo l'oggetto contenente il training dataset
train_dataset = datasets.FashionMNIST(
    '../data',          # path in cui salvo
    download=True,     
    train=True,         # prendo il training set
    transform=transforms.Compose([
        # No resize, sono già 28x28
        transforms.ToTensor(),
        transforms.Normalize((0.268), (0.353))
    ])
)

# Ora mi creo il loader per il training dataset
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
)

# Oggetto contenente i dati del test set:
test_dataset = datasets.FashionMNIST(
    '../data',  #path
    download=True,
    train=False,    # prendo solo il test
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.268), (0.353))
    ])
)

# Creo un test_set loader:
test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=1000, 
    shuffle=True
)

print(len(train_dataset), len(train_loader))

60000 938


# Addestramento della rete

In [38]:
# set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [39]:
# define train and test function
accuracy_list = []
def train(epoch, model, optimizer, perm=None):
    model.train()
    # dataloader will iterate the dataset and return images (data)
    # and labels (target)
    for batch_idx, (data, target) in enumerate(train_loader):
        # send to device
        data, target = data.to(device), target.to(device) #carico data e target sul device

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, perm=None):
    # metto il modello in evaluation(), così non avviene l'addestramento!!!
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # send to device
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        # qua anzichè definire una lista di test loss usa un accumulatore unico, perchè non dobbiamo ciclare sui batch come durante il tranining
        test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

In [40]:
model.to(device)
for epoch in range(0, 3):
    train(epoch, model, optimizer)
    test(model)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.293220
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.287865
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.260003
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.225650
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.148962
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.005921
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.723622
Train Epoch: 0 [44800/60000 (75%)]	Loss: 1.452485
Train Epoch: 0 [51200/60000 (85%)]	Loss: 1.279127
Train Epoch: 0 [57600/60000 (96%)]	Loss: 1.071867

Test set: Average loss: 1.0393, Accuracy: 6415/10000 (64%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.146665
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.961202
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.002684
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.894456
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.945238
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.973942
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.800318
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.747620
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.885011
T